In [1]:
#import packages
import os
import glob
import pandas as pd
import re

In [2]:
#Create dataframe from parliament-txt transcripts
# Run 1 time only
#dialogsDF
dialogsDF = pd.DataFrame()

#regex expressions to extract speakers
SPEAKER_REGEX = re.compile(r'[A|O] Sr\.ª?( [\w\-,]+)+([\-\w]+)?( \([\w\s\.,\-]+\))*(?=:)')
MULTILINE_SPEAKER_REGEX = re.compile("^" + SPEAKER_REGEX.pattern, flags=re.M) 
c=0
for file in os.scandir("./discursos"):
    speech_by_speaker = []

    with open(file, encoding="utf8") as fp:
        content = fp.read()
    
        #1)extract dates and numbers

        dates_txt = re.compile("\d{1,2} [A-Z]+ [A-Z]+ [A-Z]+ \d{4} (\n\n\d+)?")
        dates_txt.sub(" ", content)

        series_numb = re.compile("[A-Z]+ SÉRIE \W NÚMERO \d+ (\n+\d+)?")
        series_numb.sub(" ", content)

        deputies = MULTILINE_SPEAKER_REGEX.finditer(content) 
        deputies = [match.group() for match in deputies]
        deputies = list(set(deputies))

        content_on_one_line = ''.join(content.splitlines())

        for speaker in deputies:
            content_split_by_speaker = content_on_one_line.split(speaker + ':')
            # Get rid of everything before they speak
            content_split_by_speaker.pop(0)

            # what_they_say = [speech for speech in speaker_regex]
            # for each line in content_split_by_speaker
            # use the speaker regex to find the next speaker and get everything before that
            for line in content_split_by_speaker:
                start_of_next_speaker = SPEAKER_REGEX.finditer(line)

                # either an index or None
                # if it's None, either:
                # - the regex isn't accurate enough; or
                # - the speaker is the only person to speak]
                next_speaker_indexes = [match.start() for match in start_of_next_speaker]
                next_speaker_index = next_speaker_indexes[0] if len(next_speaker_indexes) > 0 else None

                speech = line[0:next_speaker_index] if next_speaker_index is not None else line
                speech_by_speaker.append((speaker, speech))

                # if "):" in speech:
                #     print(speech)
        
        df = pd.DataFrame.from_records(speech_by_speaker, columns=["speaker", "speech"])#.reset_index()
        df['filename'] = os.path.basename(file)
        #print(df)
                
        dialogsDF = pd.concat([dialogsDF, df])
dialogsDF.to_pickle('dialogsDF.pkl')  
        
#conjunto_dados#.to_csv("deputados.csv")#, index=False)
    

In [50]:
# load dataframe with speaches
dialogsDF = pd.read_pickle('dialogsDF.pkl')  
#see it
print(dialogsDF.shape)
dialogsDF.tail(3)

(151340, 3)


,speaker,speech,filename
114,O Sr. Ministro da Educação (Tiago Brandão Rodr...,— Obrigado!,darl14sl03n031.txt
115,O Sr. Secretário (Diogo Leão),"— Sr.ª Presidente, Srs. Deputados, relativame...",darl14sl03n031.txt
116,O Sr. Secretário (Diogo Leão),"— Sr.ª Presidente e Srs. Deputados, relativam...",darl14sl03n031.txt


In [35]:
def remove_punctuation(col_name, df):
    df[col_name]=df[col_name].str.replace('[^\w\s]', '')
    print('Punctuation removed')
    return df

def remove_special_char(col_name, df, char1,char2):
    df[col_name]=df[col_name].str.replace(char1,char2)
    print('Character '+char1 + 'replaced by '+ char2)
    return df

def aux_remove_accents(a):
    import unidecode
    return unidecode.unidecode(a)

def remove_accents(col_name, df ):
    df[col_name] = df[col_name].apply(aux_remove_accents)
    print('Accents removed')
    return df



In [51]:
# 1 Speakers selection
#remove ministers, secretaries, presidents' dialogs
dialogsDF['clean_speaker']=dialogsDF['speaker'].str.lower()
dialogsDF= remove_special_char('clean_speaker', dialogsDF, 'ç','c') #dialogsDF['clean_speaker'].str.replace('ç','c')
dialogsDF= remove_punctuation('clean_speaker', dialogsDF) #dialogsDF['clean_speaker'].str.replace('[^\w\s]', '')
dialogsDF= remove_accents('clean_speaker', dialogsDF)
dialogsDF = dialogsDF[~dialogsDF['clean_speaker'].str.contains('secretari*', na=False)]
dialogsDF = dialogsDF[~dialogsDF['clean_speaker'].str.contains('ministr*', na=False)]
dialogsDF = dialogsDF[~dialogsDF['clean_speaker'].str.contains('presidente*', na=False)]
dialogsDF.drop('clean_speaker',  axis=1, inplace=True)
# 1.1 Speakers cleaning name
aux = dialogsDF["speaker"].str.split("(", n=1, expand=True)
aux.columns = ["speaker", "party"]
dialogsDF["speaker"] = aux["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")
dialogsDF["speaker"] = dialogsDF["speaker"].str.rstrip(" ")
dialogsDF['speaker'] = dialogsDF['speaker'].str.lower()


#2 speach data cleaning
dialogsDF["speech"] = [string.lstrip("— ") for string in dialogsDF["speech"]]
# 3 creation of number columns
dialogsDF['number']=[ii[11:14] for ii in dialogsDF.filename]
dialogsDF['number']=pd.to_numeric(dialogsDF['number'],errors='coerce')
dialogsDF['session']=[ii[8:10] for ii in dialogsDF.filename]
dialogsDF['session']=pd.to_numeric(dialogsDF['session'],errors='coerce')
dialogsDF['term']=[ii[4:6] for ii in dialogsDF.filename]
dialogsDF['term']=pd.to_numeric(dialogsDF['term'],errors='coerce')

# 4 creation of party column
dialogsDF["party"] = aux["party"].str.rstrip(")")
# 5 creation of dates column for each parliament session 
dates_term=pd.read_csv('dates_terms.csv')
dates_term['number']=pd.to_numeric(dates_term['number'],errors='coerce')
dates_term=dates_term.dropna()
dialogsDF=dialogsDF.merge(dates_term, on=['term','session','number'], how='left')

Character çreplaced by c


C:\Users\trind\AppData\Local\Temp\ipykernel_868\3502243115.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col_name]=df[col_name].str.replace('[^\w\s]', '')


Punctuation removed
Accents removed


C:\Users\trind\AppData\Local\Temp\ipykernel_868\1481384319.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  dialogsDF["speaker"] = aux["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")


In [55]:
dialogsDF.head()

,speaker,speech,filename,number,session,term,party,Date
0,ferro rodrigues,"Sr. Presidente, o Grupo Parlamentar do PS indi...",darl13sl01n001.txt,1,1,13,PS,23/10/2015
1,luís montenegro,"Muito bom dia a todos, Sr.as e Srs. Deputados....",darl13sl01n001.txt,1,1,13,PSD,23/10/2015
2,luís montenegro,"O seu exemplo de tolerância, imparcialidade, d...",darl13sl01n001.txt,1,1,13,PSD,23/10/2015
3,luís montenegro,"Sr. Presidente, o Grupo Parlamentar do PSD ind...",darl13sl01n001.txt,1,1,13,PSD,23/10/2015
4,luís montenegro,"Sr. Presidente, Sr.as e Srs. Deputados: Hoje f...",darl13sl01n001.txt,1,1,13,PSD,23/10/2015


In [68]:

dialogsDF['Date'].strftime('%m-%d-%Y')


AttributeError: 'Series' object has no attribute 'strftime'

In [ ]:
'https://debates.parlamento.pt/catalogo/r3/dar/01/'+dialogsDF.term+'/'+term+'/'+number+'/'+Date